In [ ]:
import sys
sys.path.append('../')
import utility as util

config = util.load_specific_yaml('instance_F_1.yaml')

In [ ]:
# heuristic_greedy
"""
進步中 = true
最佳W = 0
已分配設施 = {...都是0}
while 此輪還在進步 and 還有工廠能蓋
    進步中 = false
    此輪最佳W 暫時= 最佳W（前一輪的）
    對每個可能的建立工廠點 j:
        看看哪個車k對於這個點的V最大 一直塞到此點容納不下車or沒車可塞了，
        剩下用decoration補滿（G最大133.4, E最大100), 此處塞滿E
        計算總W (包含先前iter已蓋好之工廠點)
        如果大於此輪最佳，記錄為此輪最佳，更新最佳蓋哪個點 j
    如果此輪最佳W > 最佳W:
        更新已分配設施用這個最佳設廠點j跟已分配設施
        更新最佳W
"""
